<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [2]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [3]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.19.2
Pandas:  1.1.3


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [5]:
# wrong: header=none
bikes = pd.read_table('dat/bikeshare.csv', sep= ',', header = None)
print(bikes.head())
print()

# wrong: row 1 becomes header
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong: header=0 correct one to bring the names of the columns (between comas)
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [2]:
#ANSWER:
bikes = pd.read_table('bikeshare.csv', header=0, sep= ',')

NameError: name 'pd' is not defined

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [5]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [6]:
pd.option('max')
pd.set_option('display.max_rows', 500)


AttributeError: module 'pandas' has no attribute 'option'

In [0]:
#make sure you include the extension .xls, sheet_name
df = pd.read_excel('data/Iris.xls', sheet_name = 'Iris')
df

So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [0]:
#ANSWER
df= pd.read_excel('file.xls', sheet_name = '', header = 0, usecols = [0,1,2,3,4])

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [7]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-01-06T09:04:05.448Z,63.2122,-151.1593,2.3,2.9,ml,NaN,NaN,NaN,0.99,...,2021-01-06T09:12:50.133Z,"46 km SE of Denali National Park, Alaska",earthquake,NaN,0.4,NaN,NaN,automatic,ak,ak


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:


In [8]:
#unstructured data, 
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df[0]

ValueError: No tables found

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [11]:
#more like dictionaries
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [12]:
#ANSWER (rows, columns)
bikes.shape


NameError: name 'bikes' is not defined

What are the column names in `bikes`?

In [1]:
#ANSWER:all the names of the columns
bikes.columns

NameError: name 'bikes' is not defined

What are the data types of these columns?

In [0]:
#ANSWER
bikes.columns
bikes.head()

What is the (row) index for this DataFrame?

In [0]:
#ANSWER
bikes.index

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [0]:
#ANSWER
print(bikes['atemp'].head())
print(type(bikes['atemp']))
print(type(bikes[['atemp']]))
(bikes[['atemp'].dtypes

How would we use object notation to show the first 4 rows of `atemp`?

In [0]:
#ANSWER
print(bikes.atemp[:4]bikes.atemp[2:4]

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [0]:
bikes[[0]] #subseting a column with name 0 and it does not exist
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

In [ ]:
bikes.head()


What is the correct way to access the 1st row of the DataFrame by its index?

In [0]:
#ANSWER
bikes[:1]
bikes[:0] #error because no column zero

What is the correct way to access the 2nd column of the DataFrame by its index?

In [0]:
#ANSWER
bikes.iloc[:,1] #number
bikes.loc['season'] #strings


## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [0]:
bikes.isnull() #null value False(0)/True(1)
bikes.isnull().head() 
bikes.isnull().sum() #counting 0 & 1
bikes['windspeed'].isnull().sum()

However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [0]:
#ANSWER


If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [0]:
#ANSWER


What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [0]:
#ANSWER


Apply the Pandas `isna` function to the following data objects:

In [0]:
x = 2.3
y = np.nan
print(x, y)

In [0]:
#ANSWER


In [0]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

In [0]:
#ANSWER


How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [0]:
#ANSWER
bikes['season'].value_counts()
bikes.weekday.value_counts()
bikes.mnth.value_counts()

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [10]:
#ANSWER
print('instant', bikes['instant'].min(), 'to', bikes['instant'],max())

TypeError: max expected 1 argument, got 0

Compute and print the overall minimum and maximum of the numeric data columns:

In [9]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max

TypeError: '<' not supported between instances of 'str' and 'int'

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [0]:
bikes['atemp'].quantile(0.5)

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [0]:
#ANSWER
bikes['atemp']quantile((0.1, 0.25, 0.5, 0.75, 0.9))

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [0]:
type(bikes['atemp'])

In [0]:
bikes.sample(5)

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [0]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [0]:
#ANSWER


Here is a random sample of `atemp_level`:

In [0]:
atemp_level.sample(5)          

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [0]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [0]:
#ANSWER


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [0]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [0]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [ ]:
rock

## Check Column Names

Check column names and clean.

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [ ]:
rock.isnull().sum()

In [ ]:
#Create amask to find all rows with null values
null_mask=rock['release_year'].isnull()
rock.loc[null_mask, 'release_year'] = 0
rock.isnull().sum()

In [ ]:
rock['release_year'].fillna(0)

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [ ]:
#check datatypes
rock.info()
rock.dtypes

In [ ]:
song_facts_mask = rock['release_year'] == "SONGFACTS.COM"
rock.loc[song_facts_mask, 'release_year'] = np.NaN

In [ ]:
#convert realseyear to numeric
rock['release_year'] = pd.to_numeric(rock['release_year'])

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.



> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



